# Projeto Semantix

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time

sns.set_theme()

%reload_ext watermark
%watermark -iv -v

import warnings

warnings.filterwarnings("ignore")

C:\Users\Marcelo\AppData\Local\Temp\ipykernel_980\3125395472.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Python implementation: CPython
Python version       : 3.12.2
IPython version      : 8.21.0

numpy     : 1.26.4
pandas    : 2.2.0
seaborn   : 0.13.2
matplotlib: 3.8.3



In [3]:
df = pd.read_csv('./Dados/matches.csv')
df.drop(columns='Notes', inplace=True)
df

,Unnamed: 0,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,...,Referee,Match Report,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
0,0,2023-08-13,16:30,Premier League,Matchweek 1,Sun,Away,D,1.0,1.0,...,Anthony Taylor,Match Report,13.0,1.0,17.8,0.0,0,0,2024,Liverpool
1,1,2023-08-19,15:00,Premier League,Matchweek 2,Sat,Home,W,3.0,1.0,...,Thomas Bramall,Match Report,25.0,9.0,16.8,1.0,0,1,2024,Liverpool
2,2,2023-08-27,16:30,Premier League,Matchweek 3,Sun,Away,W,2.0,1.0,...,John Brooks,Match Report,9.0,4.0,17.2,1.0,0,0,2024,Liverpool
3,3,2023-09-03,14:00,Premier League,Matchweek 4,Sun,Home,W,3.0,0.0,...,Simon Hooper,Match Report,17.0,4.0,14.7,0.0,0,0,2024,Liverpool
4,4,2023-09-16,12:30,Premier League,Matchweek 5,Sat,Away,W,3.0,1.0,...,Michael Oliver,Match Report,16.0,5.0,15.8,0.0,0,0,2024,Liverpool
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,20,2023-12-30,15:00,Premier League,Matchweek 20,Sat,Away,L,0.0,2.0,...,David Coote,Match Report,4.0,2.0,14.9,0.0,0,0,2024,SheffieldUnited
472,22,2024-01-21,14:00,Premier League,Matchweek 21,Sun,Home,D,2.0,2.0,...,Michael Salisbury,Match Report,20.0,5.0,18.5,1.0,1,1,2024,SheffieldUnited
473,24,2024-01-30,20:00,Premier League,Matchweek 22,Tue,Away,L,2.0,3.0,...,Tony Harrington,Match Report,9.0,5.0,16.5,0.0,0,0,2024,SheffieldUnited
474,25,2024-02-03,17:30,Premier League,Matchweek 23,Sat,Home,L,0.0,5.0,...,Paul Tierney,Match Report,10.0,4.0,10.0,0.0,0,0,2024,SheffieldUnited


In [4]:
df.iloc[:,0:20].sort_values(by='Date', ascending=False)

,Unnamed: 0,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Sh
238,32,2024-02-12,20:00,Premier League,Matchweek 24,Mon,Away,W,3.0,1.0,Crystal Palace,1.4,0.6,77.0,25110.0,Ben Chilwell,4-2-3-1,Michael Oliver,Match Report,14.0
356,27,2024-02-12,20:00,Premier League,Matchweek 24,Mon,Home,L,1.0,3.0,Chelsea,0.6,1.4,23.0,25110.0,Joachim Andersen,4-1-4-1,Michael Oliver,Match Report,13.0
70,33,2024-02-11,14:00,Premier League,Matchweek 24,Sun,Away,W,6.0,0.0,West Ham,3.6,0.2,70.0,62475.0,Martin Ødegaard,4-3-3,Craig Pawson,Match Report,24.0
118,35,2024-02-11,16:30,Premier League,Matchweek 24,Sun,Home,L,1.0,2.0,Manchester Utd,2.4,1.8,56.0,42185.0,John McGinn,4-2-3-1,Robert Jones,Match Report,23.0
142,33,2024-02-11,16:30,Premier League,Matchweek 24,Sun,Away,W,2.0,1.0,Aston Villa,1.8,2.4,44.0,42185.0,Bruno Fernandes,4-2-3-1,Robert Jones,Match Report,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,0,2023-08-12,17:30,Premier League,Matchweek 1,Sat,Home,W,5.0,1.0,Aston Villa,3.3,1.8,52.0,52207.0,Kieran Trippier,4-3-3,Andy Madley,Match Report,17.0
95,0,2023-08-12,17:30,Premier League,Matchweek 1,Sat,Away,L,1.0,5.0,Newcastle Utd,1.8,3.3,48.0,52207.0,John McGinn,4-2-3-1,Andy Madley,Match Report,16.0
404,0,2023-08-12,15:00,Premier League,Matchweek 1,Sat,Home,L,0.0,1.0,Fulham,2.7,1.5,42.0,39940.0,James Tarkowski,4-4-1-1,Stuart Attwell,Match Report,19.0
428,0,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Home,L,0.0,3.0,Manchester City,0.3,1.9,35.0,21572.0,Josh Cullen,5-4-1,Craig Pawson,Match Report,6.0
